In [ ]:
import h5py
import numpy as np

with h5py.File('dataset.h5', 'r') as f:
    X_train = f['X_train'][:]
    X_val = f['X_val'][:]
    y_train = f['y_train'][:]
    y_val = f['y_val'][:]

print("Loaded preprocessed data:")
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)

Loaded preprocessed data:
X_train shape: (288, 30, 1, 1, 2048)
X_val shape: (72, 30, 1, 1, 2048)
y_train shape: (288,)
y_val shape: (72,)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau 
from sklearn.metrics import roc_auc_score
from tensorflow.keras.layers import Input

gpus = tf.config.experimental.list_physical_devices('GPU')

INPUT_SIZE = (299, 299)
FEATURE_DIM = 2048
NUM_FRAMES = 30
TRAIN_REAL = 'extracted_frames/train/real'
TRAIN_FAKE = 'extracted_frames/train/fake'

In [3]:
def build_convlstm_model(input_shape=(NUM_FRAMES, 1, 1, FEATURE_DIM)):
    model = Sequential([
        ConvLSTM2D(filters=64, kernel_size=(1, 1), input_shape=input_shape,
                   return_sequences=False,
                   padding='same',
                   recurrent_activation='hard_sigmoid',
                   activation='tanh'),
        BatchNormalization(),
        Flatten(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

model = build_convlstm_model(input_shape=(NUM_FRAMES, 1, 1, FEATURE_DIM))
model.summary()

C:\Users\Ishita\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_lstm2d (ConvLSTM2D)        │ (None, 1, 1, 64)       │       540,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1, 1, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 545,409 (2.08 MB)

 Trainable params: 545,281 (2.08 MB)

 Non-trainable params: 128 (512.00 B)

In [4]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.001,
    verbose=1
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=8,
    epochs=30, 
    callbacks=[reduce_lr],
    verbose=1
)

Epoch 1/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 136ms/step - accuracy: 0.4989 - loss: 0.9055 - val_accuracy: 0.6250 - val_loss: 0.6544 - learning_rate: 0.0010
Epoch 2/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 126ms/step - accuracy: 0.6474 - loss: 0.6993 - val_accuracy: 0.6111 - val_loss: 0.6337 - learning_rate: 0.0010
Epoch 3/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 110ms/step - accuracy: 0.6801 - loss: 0.5914 - val_accuracy: 0.6528 - val_loss: 0.6334 - learning_rate: 0.0010
Epoch 4/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 106ms/step - accuracy: 0.7437 - loss: 0.5168 - val_accuracy: 0.6250 - val_loss: 0.6256 - learning_rate: 0.0010
Epoch 5/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.7696 - loss: 0.4847 - val_accuracy: 0.5694 - val_loss: 0.6546 - learning_rate: 0.0010
Epoch 6/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 118ms/step - accuracy: 0.7516 - loss: 0.4925 - val_accuracy: 0.6667 - val_loss: 0.5881 - learning_rate: 0.0010
Epoch 7/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 203ms/step - accuracy: 0.7319 - loss: 0.5140 -

In [5]:
y_pred_prob = model.predict(X_val).flatten()
y_pred = (y_pred_prob > 0.5).astype(int)

auc_score = roc_auc_score(y_val, y_pred_prob)
print(f"AUC Score: {auc_score:.4f}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 321ms/step
AUC Score: 0.7755
